Source:

FOCUS ON CAPACITY 16

In [1]:
!pip install pyswarm

  Preparing metadata (setup.py) ... done
  Created wheel for pyswarm: filename=pyswarm-0.6-py3-none-any.whl size=4463 sha256=a978ad7db550c7542ba284971475d05907742112fc2fc395be9d194f911485ff
  Stored in directory: /root/.cache/pip/wheels/71/67/40/62fa158f497f942277cbab8199b05cb61c571ab324e67ad0d6
Successfully built pyswarm


In [2]:
#import library
import pandas as pd
import numpy as np
import math
import random
#for harvisine function -calculate distance
from math import radians, cos, sin, asin, sqrt
#for plotting
import matplotlib.pyplot as plt

In [3]:
# Define the CVRP problem parameters
num_customers = 51  # Number of customers (excluding depot)
num_vehicles = 5  # Number of vehicles
vehicle_capacity = 16  # Capacity of each vehicle

In [4]:
# PSO parameters
num_particles = 50 # Number of particles in the swarm
max_iterations = 200  # Maximum number of iterations
inertia_weight = 0.7  # Inertia weight
cognitive_weight = 2.05 # Cognitive weight
social_weight = 2.05 # Social weight

Part 1: Read Dataset

In [5]:
# Read the dataset from CSV
data = pd.read_csv('STESEN MINYAK KL_P.csv')
data.head()


,index,city,lat,long,demand,vehicle_cap
0,0,Kuala Lumpur,2.95980,101.662,0,16
1,1,Kuala Lumpur,3.04885,101.680,15,16
2,2,Kuala Lumpur,3.05643,101.704,10,16
3,3,Kuala Lumpur,3.06578,101.663,11,16
4,4,Kuala Lumpur,3.06669,101.656,7,16


In [6]:
# Extract the coordinates, demands, and vehicle capacity
coordinates = data[['lat', 'long']].values[:num_customers + 1]  # Include depot as the first customer
demands = data['demand'].values[:num_customers]
vehicle_capacities = np.full(num_vehicles, vehicle_capacity)


In [7]:
# define the Haversine formula to calculate the distance between two points
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # radius of the Earth in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    return R * c

# calculate the distance matrix for the points using the Haversine formula
distance_matrix = np.zeros(shape=(len(coordinates), len(coordinates)), dtype=float)
for i in range(len(coordinates)):
    for j in range(len(coordinates)):
        distance_matrix[i][j] = haversine(coordinates[i][0], coordinates[i][1],coordinates[j][0], coordinates[j][1])

print(distance_matrix)

[[ 0.         10.10162453 11.71326462 ... 29.56603256 30.41708417
  30.03857492]
 [10.10162453  0.          2.79500543 ... 20.07651663 20.33278341
  20.36793443]
 [11.71326462  2.79500543  0.         ... 20.05430901 18.95293159
  20.14745823]
 ...
 [29.56603256 20.07651663 20.05430901 ...  0.          9.99176737
   1.44443894]
 [30.41708417 20.33278341 18.95293159 ...  9.99176737  0.
   8.67507083]
 [30.03857492 20.36793443 20.14745823 ...  1.44443894  8.67507083
   0.        ]]


In [8]:
# Define the Particle class
class Particle:
    def __init__(self, position):
        self.position = position
        self.velocity = np.zeros_like(position)
        self.personal_best_position = np.copy(position)
        self.personal_best_fitness = float('inf')

In [9]:
# Initialize the swarm
swarm = []
for _ in range(num_particles):
    position = np.random.permutation(num_customers) + 1  # Generate a random permutation of customers
    particle = Particle(position)
    swarm.append(particle)

In [10]:
# Initialize the global best position and fitness
global_best_position = 0
global_best_fitness = float('inf')


In [11]:
# Calculate the fitness value for a particle's position

def calculate_fitness(position,distance):
    total_distance = 0

    remaining_capacities = np.copy(vehicle_capacities)
    current_location = 0  # Start at the depot
    for customer in position:
        distance = distance[current_location][customer]
        total_distance += distance
        remaining_capacities -= demands[customer - 1]  # Subtract the demand of the customer
        current_location = customer
    distance_back_to_depot = distance[current_location][0]  # Return to the depot
    total_distance += distance_back_to_depot
    max_capacity_violation = np.max(remaining_capacities)
    fitness = total_distance + max_capacity_violation  # Objective function combining distance and capacity violation

    return (distance)

In [12]:
# Retrieve the best solution found by PSO
best_position = global_best_position

# Calculate total distance traveled and capacity utilization
if best_position != 0:
  total_distance = calculate_fitness(best_position)
  capacity_utilization = 1 - (total_distance - num_customers) / (num_customers * vehicle_capacity)

# Print the outputs
  print("Minimum Total Distance Traveled:", total_distance)
  print("Maximum Capacity Utilization:", capacity_utilization)

# Obtain the routing plan for each vehicle
  vehicle_routes = [[] for _ in range(num_vehicles)]
  current_location = 0  # Start at the depot
  for customer in best_position:
     if customer == 0:  # Reached the depot, move to the next vehicle
        current_location = 0
     else:
        vehicle_routes[current_location].append(customer)
        current_location = customer

# Print the routing plan for each vehicle
  for i, route in enumerate(vehicle_routes):
    print("Vehicle", i+1, ":", route)

else:
  print("no solution found")


no solution found


In [14]:
distance=[]
import matplotlib.pyplot as plt


# Initialize a list to track the global best fitness value at each iteration
global_best_fitness_history = []

# Initialize global best position
global_best_position = 0

# Initialize the PSO algorithm
for iteration in range(max_iterations):
    # Update the velocity and position of each particle
    for particle in swarm:
        # Update particle velocity
        particle.velocity = (inertia_weight * particle.velocity +
                              cognitive_weight * random.random() * (particle.personal_best_position - particle.position) +
                              social_weight * random.random() * (global_best_position - particle.position))

        # Update particle position
        particle.position = np.clip(particle.position + particle.velocity, 1, num_customers)  # Ensure valid positions

        # Update particle personal best position and fitness
        particle_fitness = calculate_fitness(particle.position,distance)
          if particle_fitness < particle.personal_best_fitness:
             particle.personal_best_position = np.copy(particle.position)
             particle.personal_best_fitness = particle_fitness

        # Update the global best position and fitness
          if global_best_position is None or particle_fitness < objective_function(global_best_position):
            global_best_position = np.copy(particle.personal_best_position)

    # Track the global best fitness at each iteration
    global_best_fitness = calculate_fitness(global_best_position)
    global_best_fitness_history.append(global_best_fitness)

# Plot the convergence
plt.plot(range(max_iterations), global_best_fitness_history)
plt.xlabel('Iteration')
plt.ylabel('Global Best Fitness')
plt.title('PSO Convergence')
plt.show()

IndentationError: ignored